In [ ]:
# === Import core libraries and enable future compatibility ===
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
from sympy import integrate, exp, sin, log, oo, pi,symbols
import deepxde as dde
import scipy.io as scio
from mpl_toolkits.mplot3d import Axes3D
from deepxde.backend import tf
#import datetime
# import time
import os
from pylab import mpl
from scipy.signal import chirp, spectrogram
mpl.rcParams['font.sans-serif']=['Microsoft YaHei']
mpl.rcParams['axes.unicode_minus']=False
from matplotlib.pyplot import MultipleLocator
# import xlrd
# import xlwt
from sympy import *
import sympy as sp
# import torch

In [ ]:
pip install sympy

In [ ]:
# === Configure TensorFlow environment flags for JIT and log level ===
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
tf.function(jit_compile=True)

In [ ]:
# === Define function to round numbers to given significant figures ===
def round_sf(x, significant_figure=3):
    fmt = "%%.%dg" % significant_figure
    if isinstance(x, np.ndarray):
        return np.array([float(fmt % i) for i in x])
    else:
        return float(fmt % x)

In [ ]:
#Read the direct output of the network
def Output(X):
    Out_final = np.zeros((X.shape[0],7))
    import re
    lines = open("s1.dat", "r").readlines()
    S = np.array([np.fromstring(min(re.findall(re.escape("[") + "(.*?)" + re.escape("]"), line),\
                                key=len),sep=",",) for line in lines ])
    dis = S[:,0:num_dis+2][-1]
    dis2 = np.array([-0.2,L/2+L_dis/2,1.7])
    dis[0] = -0.2
    dis[-1] = 1.7
    x = X[:,0:1]
    index  = [ np.where((x>=dis[i])&(x<dis[i+1]))[0] for i in range(0,dis.shape[0]-1) ]
    index2 = [ np.where((x>=dis2[i])&(x<dis2[i+1]))[0] for i in range(0,dis2.shape[0]-1) ]
    interv = [ (x>=dis[i])&(x<dis[i+1]) for i in range(0,dis.shape[0]-1) ]
    interv2 = [ (x>=dis2[i])&(x<dis2[i+1]) for i in range(0,dis2.shape[0]-1) ]
    out_all = model.predict(X)
    
    u_fai = out_all[:,0:4]
    for i in range(2):
        Temp = u_fai[:,i::2][interv2[i].ravel()]
        Out_final[:,0:2][index2[i]] = Temp
                     
    Y = out_all[:, 4: 4*num_NN+1]
    for i in range(0,num_NN):
        Temp = Y[:,i::num_NN][interv[i].ravel()]
        Out_final[:,2:-2][index[i]] =  Temp
    Out_final[:,-1:] = out_all[:, -1:]
    Out_final[:,-2:-1] = out_all[:, -2:-1]
    return [Out_final[:,i:i+1] for i in range(0,7)]

#Read the output of the derivative function of the network
def Output_dNN(X, j = 0):
    Y = np.zeros((X.shape[0],1))
    import re
    lines = open("s1.dat", "r").readlines()
    S = np.array([np.fromstring(min(re.findall(re.escape("[") + "(.*?)" + re.escape("]"), line),\
                                key=len),sep=",",) for line in lines ])
    dis = S[:,0:num_dis+2][-1]
    dis2 = np.array([-0.2,L/2+L_dis/2,1.7])
    dis[-1] = dis[-1]+0.001
    dis[0] = -0.2
    dis[-1] = 1.7
    x = X[:,0:1]
    index  = [ np.where((x>=dis[i])&(x<dis[i+1]))[0] for i in range(0,dis.shape[0]-1) ]
    index2 = [ np.where((x>=dis2[i])&(x<dis2[i+1]))[0] for i in range(0,dis2.shape[0]-1) ]
    interv = [ (x>=dis[i])&(x<dis[i+1]) for i in range(0,dis.shape[0]-1) ]
    interv2 = [ (x>=dis2[i])&(x<dis2[i+1]) for i in range(0,dis2.shape[0]-1) ]
    if j == 0 or j == 1:
        for i in range(2):
            Temp = model.predict(X[interv2[i].ravel()], operator=lambda x,y:dde.grad.jacobian(y,x, i= j*2+i,j=0))
            Y[index2[i]] = Temp
    else:
        for i in range(0,num_NN):
            Temp = model.predict(X[interv[i].ravel()], operator=\
                                                   lambda x,y:dde.grad.jacobian(y,x, i= 4 + (j-2)*num_NN + i,j=0))
            Y[index[i]] = Temp
    if j == 4 or j ==0:
        Y = -Y
    return Y

#The L2 norm error between the output value directly obtained from the network and the derivative function of the network
def Norm_NN(X):
    W,fai,dfai,M,V,Dy_t,EI= Output(X)
    fai_dNN,dfai_dNN,V_dNN = [Output_dNN(X, j = i) for i in [0,1,3]]

    M_dNN = dfai*EI
    pre_y1,pre_y2 = Output_label()
    EI_exact = NP_EI(X[:,0:1])

    norm_fai_dNN = np.linalg.norm(fai-fai_dNN,ord=2) / np.linalg.norm(fai_dNN,ord=2)
    norm_dfai_dNN = np.linalg.norm(dfai-dfai_dNN,ord=2) / np.linalg.norm(dfai_dNN,ord=2)
    norm_M_dNN = np.linalg.norm(M-M_dNN,ord=2) / np.linalg.norm(M_dNN,ord=2)
    norm_V_dNN = np.linalg.norm(V-V_dNN,ord=2) / np.linalg.norm(V_dNN,ord=2)
    norm_y1 = np.linalg.norm(ob_y1-pre_y1,ord=2) / np.linalg.norm(ob_y1,ord=2)
    norm_y2 = np.linalg.norm(ob_y2-pre_y2,ord=2) / np.linalg.norm(ob_y2,ord=2)
    Norm = np.hstack((norm_fai_dNN,norm_dfai_dNN,norm_M_dNN,norm_V_dNN,1,1,norm_y1,norm_y2))
    print('norm_fai_dNN,norm_dfai_dNN,norm_M_dNN,norm_V_dNN,--,--,norm_y1,norm_y2')
    print(Norm)
    return Norm

#The predicted value of the point where the output label data is located
def Output_label():
    pre_y0 = (model.predict(ob_x1)[:,0:1]).reshape(-1,1)
    pre_y1 = (model.predict(ob_x2)[:,1:2]).reshape(-1,1)
    Pre_y  = np.vstack((pre_y0,pre_y1))
    return pre_y0,pre_y1

In [ ]:
def Output_dy_tt(X):
    j = 0
    dis = np.array([-0.2,L/2+L_dis/2,1.7])
    x = X[:,0:1]
    interv = [ (x>=dis[i])&(x<dis[i+1]) for i in range(0,dis.shape[0]-1) ]
    Out_dNN = model.predict(X[interv[0].ravel()], operator=lambda x,y:dde.grad.hessian(y,x,component=j, i=1,j=1))
    for i in range(1,2):
        Out_dNN = np.vstack((Out_dNN,model.predict(X[interv[i].ravel()], operator=\
                                               lambda x,y:dde.grad.hessian(y,x,component=j+i, i=1,j=1))))
    return Out_dNN

def Output_dy_t(X):
    j = 0
    dis = np.array([-0.2,L/2+L_dis/2,1.7])
    x = X[:,0:1]
    interv = [ (x>=dis[i])&(x<dis[i+1]) for i in range(0,dis.shape[0]-1) ]
    Out_dNN = model.predict(X[interv[0].ravel()], operator=lambda x,y:dde.grad.jacobian(y,x, i=j,j=1))
    for i in range(1,2):
        Out_dNN = np.vstack((Out_dNN,model.predict(X[interv[i].ravel()], operator=\
                                               lambda x,y:dde.grad.jacobian(y,x, i=j+i, j=1))))
    return Out_dNN

In [ ]:
#Read the true value
file_name='beam_dynamic_5HZ_VM_6mm.mat'
def Solution(x=0.15,t=-1,name='Dis'):
    data = scio.loadmat(file_name)
    exact = data[name]
    T = data["T"][0][0]
    L_Ne = (data["L"]/data["Ne"])[0][0]
    dt = data["dt"][0][0]
    if t == -1:  # Return the time history of the deflection of the vibration at point x
        index = round(x/L_Ne)
        T = np.arange(0,T+dt,dt).reshape(-1,1)
        X = np.ones_like(T)*x
        return np.hstack((X,T,exact[index,:].reshape(-1,1)))
    if x == -1: #The vibration mode of the deflection at time t
        X = np.arange(0,L+L_Ne,L_Ne).reshape(-1,1)
        T =  np.ones_like(X)*t
        index = round(t/dt)
        return np.hstack((X,T,exact[:,index].reshape(-1,1)))

#Set label data
def Observe_u(X = [round(0.25 * i,2) for i in range(1,6)], T = -1):
    list_reslut = []
    if T == -1:
        for i in X:
            list_reslut.append(Solution(x=i , t=-1)[1:])
    if X == -1:
        for i in T:
            list_reslut.append(Solution(X=-1, t=i)[1:])
    return np.array(list_reslut).reshape(-1,3)

In [ ]:
#Adaptive sampling method based on residuals
def RAR(X,err_total,err,P = 0):
    list_pioint_rar = []
    for i in range(6):
        err_eq = (err_total[i]/err[i])
        x_ids = np.argsort(-err_eq,axis=0 )[:20]
        points = X[x_ids].reshape(-1,2)
        list_pioint_rar.append(points)
    if P == 1:
        return list_pioint_rar
    else:
        return np.array(list_pioint_rar).reshape(-1,2)

In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt
 
from matplotlib import rcParams
 
config = {
            "font.family": 'serif',
            "font.size": 10.5,
            "mathtext.fontset": 'stix',
            "font.serif": ['MicroSoft YaHei'],
         }
rcParams.update(config)

In [ ]:
#Define structural parameters
rho = 1180
A = 0.05*0.05
EI_real = 3e9*0.05*0.05**3/12
F = -20*1.5
w0 = 5*2*np.pi

L = 1.5
T = 0.2
L_dis = 0.006

# === Define segmentation variables (discontinuity points in structure) ===
s0 = tf.Variable(-0.01, trainable=False, dtype=tf.float32)
s1 = tf.Variable(L/2-L_dis/2, trainable=False, dtype=tf.float32)
s2 = tf.Variable(L/2+L_dis/2, trainable=False, dtype=tf.float32)
s3 = tf.Variable(L+0.01, trainable=False, dtype=tf.float32)

s_mid = tf.Variable(L/2, trainable=False, dtype=tf.float32)

SS = [s0,s1,s2,s3]
SS_mid = [s0,s_mid,s3]

#Set the scaling factor of the network
P0 = (tf.Variable(0.01, trainable=True, dtype=tf.float32))
P1 = (tf.Variable(1, trainable=True, dtype=tf.float32))
P2 = (tf.Variable(1, trainable=True, dtype=tf.float32))
P3 = (tf.Variable(1, trainable=True, dtype=tf.float32))
P4 = (tf.Variable(1, trainable=True, dtype=tf.float32))
P5 = (tf.Variable(1, trainable=True, dtype=tf.float32))
P6 = (tf.Variable(1, trainable=True, dtype=tf.float32))
P7 = (tf.Variable(1, trainable=True, dtype=tf.float32))
P8 = (tf.Variable(1, trainable=True, dtype=tf.float32))
P9 = (tf.Variable(1, trainable=True, dtype=tf.float32))
P10 = (tf.Variable(1, trainable=True, dtype=tf.float32))
# P11 = (tf.Variable(0.01, trainable=True, dtype=tf.float32))
# P12 = (tf.Variable(0.01, trainable=True, dtype=tf.float32))

#The coefficients of the adaptive activation function
c1 = tf.sigmoid(tf.Variable(0.5, trainable=True, dtype=tf.float32))
c2 = tf.sigmoid(tf.Variable(0.5, trainable=True, dtype=tf.float32))
c3 = tf.sigmoid(tf.Variable(0.5, trainable=True, dtype=tf.float32))

List_P = [P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,c1,c2,c3]

num_dis = len(SS)-2
num_NN = num_dis+1

#Define label data
ob_XTY = Observe_u()
values = ob_XTY[:, 0:1]
bins = [L/2-L_dis/2,L/2+L_dis/2]  
indices = np.digitize(values, bins)
array1 = ob_XTY[indices.flatten() == 0]
array2 = ob_XTY[indices.flatten() == 1]
array3 = ob_XTY[indices.flatten() == 2]
ob_x1,ob_y1 = array1[:,0:2],array1[:,2:3]
ob_x2,ob_y2 = array2[:,0:2],array2[:,2:3]
ob_x3,ob_y3 = array3[:,0:2],array3[:,2:3]
ob_x1,ob_y1 = Observe_u(X = [round(0.25 * i,2) for i in range(1,4)], T = -1)[:,0:2],\
                Observe_u(X = [round(0.25 * i,2) for i in range(1,4)], T = -1)[:,2:3]
# ob_x2,ob_y2 = Observe_u(X = [round(0.25 * i,2) for i in range(2,5)], T = -1)[:,0:2],\
#                 Observe_u(X = [round(0.25 * i,2) for i in range(2,5)], T = -1)[:,2:3]
ob_x3,ob_y3 = Observe_u(X = [round(0.25 * i,2) for i in range(4,6)], T = -1)[:,0:2],\
                Observe_u(X = [round(0.25 * i,2) for i in range(4,6)], T = -1)[:,2:3]
num_label = 0
# observe_u1 = dde.PointSetBC(ob_x1,ob_y1,component=num_NN*num_label)
# observe_u2 = dde.PointSetBC(ob_x2,ob_y2,component=num_NN*num_label+1)
# observe_u3 = dde.PointSetBC(ob_x3,ob_y3,component=num_NN*num_label+2)

observe_u1 = dde.PointSetBC(ob_x1,ob_y1,component=0)
observe_u3 = dde.PointSetBC(ob_x3,ob_y3,component=1)

XY0 = Solution(x=0.75,t = -1)
XY1 = Solution(x=-1,t = 0.05)
X1 = XY1[:,0:1]
T1 = XY1[:,1:2]

In [ ]:
L/2-L_dis/2

In [ ]:
#Add training anchor points
list_x_points = []
list_x_points2 = []
for i in range(1,9):    
    list_x_points.append(np.hstack((X1[::10],X1[::10]*0+0.025*i)))
    list_x_points2.append(np.hstack((X1,X1*0+0.025*i)))
list_x_points.append(np.hstack((X1[::10],X1[::10]*0+0.01)))
list_x_points2.append(np.hstack((X1,X1*0+0.01)))
anchors_x0_x = np.vstack([list_x_points[i] for i in range(0,len(list_x_points))])
anchors_x0_x0 = np.vstack([list_x_points2[i] for i in range(0,len(list_x_points2))])

K = 4
np.random.seed(K)
tf.set_random_seed(K)
anchors_x = np.vstack((np.random.uniform(0,L,8000),np.random.uniform(0,T+0.001,8000))).T
anchors_x0 = np.vstack((np.random.uniform(L/2-L_dis/2,L/2+L_dis/2,200),np.random.uniform(0,T+0.001,200))).T
anchors_x1 = np.vstack((np.linspace(L/2-L_dis/2,L/2-L_dis/2,800),np.random.uniform(0,T+0.001,800))).T
anchors_x2 = np.vstack((np.linspace(L/2+L_dis/2,L/2+L_dis/2,800),np.random.uniform(0,T+0.001,800))).T
anchors_x3 = np.vstack((np.linspace(0,0,800),np.random.uniform(0,T+0.001,800))).T
anchors_x4 = np.vstack((np.linspace(L,L,800),np.random.uniform(0,T+0.001,800))).T

anchors_x = np.vstack((anchors_x,anchors_x0))
plt.scatter(anchors_x[:,0:1],anchors_x[:,1:2],s=2)
np.savetxt('anchors_x',anchors_x)
anchors_x[0:2]

In [ ]:
#Hyperparameters and network settings
num_domain = 0
epochs = 10000
lr = 1e-3
u_nodenum=20
u_layer=2
E_nodenum=20
E_layer=2
activation_func1 = tf.nn.tanh
activation_func2 = tf.nn.tanh

rho = 1180
A = 0.05*0.05
EI_real = 3e9*0.05*0.05**3/12
F = -20*1.5
w0 = 5*2*np.pi

#External load function
def f(x_in):
    cond1 = tf.logical_and(tf.greater_equal(x_in, L/2-L_dis/2), tf.less_equal(x_in, L/2+L_dis/2))
    f1 = F/L_dis * tf.ones_like(x_in)
    f2 = tf.zeros_like(x_in)
    f = tf.where(cond1, f1, f2)
    return f

def NP_f(x_in,t_in):
    cond1 = np.logical_and(np.greater_equal(x_in, L/2-L_dis/2), np.less(x_in, L/2+L_dis/2))
    f1 = F/L_dis*np.ones_like(x_in)*np.sin(w0*t_in)
    f2 = np.zeros_like(x_in)
    f = np.where(cond1, f1, f2)
    return f

#A function for suppressing the spread of pathological information at the boundaries
def f_BC(x_in):
    cond1 = tf.logical_and(tf.greater(x_in, 0.05), tf.less(x_in, 1.45))
    f1 = tf.ones_like(x_in)
    f2 = tf.zeros_like(x_in)
    f = tf.where(cond1, f1, f2)
    return f

def f_T(t_in):
    cond1 = tf.logical_and(tf.greater(t_in, 0),tf.less(t_in, 0.01))
    f1 = tf.ones_like(t_in)
    f2 = tf.zeros_like(t_in)
    f = tf.where(cond1, f1, f2)
    return f

# === Define region-wise indicator function f_sig0(i) ===
def f_sig0(x_in,i):
    cond1 = tf.logical_and(tf.greater_equal(x_in, SS[i]), tf.less_equal(x_in, SS[i+1]))
    f1 = tf.ones_like(x_in)
    f2 = tf.zeros_like(x_in)
    f = tf.where(cond1, f1, f2)
    return f

def f_sig_mid(x_in,i):
    cond1 = tf.logical_and(tf.greater_equal(x_in, SS_mid[i]), tf.less(x_in, SS_mid[i+1]))
    f1 = tf.ones_like(x_in)
    f2 = tf.zeros_like(x_in)
    f = tf.where(cond1, f1, f2)
    return f

#Stiffness function
def TF_EI(x_in):
    f1 = EI_real*(x_in+1)/2
    return f1
def NP_EI(x_in):
    f1 = (0.75+(x_in/L-0.5)**2)
    return f1

#Set the first-order coupled PDE residual
def beampde2(x, y):
    x_in = x[:,0:1]
    t_in = x[:,1:2]
    u     = y[:,0:2]
    Fai   = y[:,2:4]
    Dfai  = y[:,1+num_NN*1:1+num_NN*2]
    M     = y[:,1+num_NN*2:1+num_NN*3]
    V     = y[:,1+num_NN*3:1+num_NN*4]
    Dy_t  = y[:,1+num_NN*4:2+num_NN*4]
    EI    = y[:,2+num_NN*4:3+num_NN*4]
    Fai_dNN = []
    Dfai_dNN = []
    M_dNN = []
    V_dNN = []
    V2_dNN = []
    Q_dNN = []
    Dy_t_dNN = []
    Dy_tt_dNN = []
    
    fai_dNN = -dde.grad.jacobian(u, x, i = 0, j=0) * f_sig_mid(x_in,0) -dde.grad.jacobian(u, x, i = 1, j=0) * f_sig_mid(x_in,1)
    dfai_dNN = dde.grad.jacobian(Fai, x, i = 0, j=0) * f_sig_mid(x_in,0) + dde.grad.jacobian(Fai, x, i = 1, j=0) * f_sig_mid(x_in,1)
    dy_t_dNN = dde.grad.jacobian(u,x, i = 0, j = 1) * f_sig_mid(x_in,0) + dde.grad.jacobian(u,x, i = 1, j = 1) * f_sig_mid(x_in,1)
    dy_tt_dNN = dde.grad.hessian(u,x,component=0, i = 1, j = 1) * f_sig_mid(x_in,0)\
                + dde.grad.hessian(u,x,component=1, i = 1, j = 1)* f_sig_mid(x_in,1)
    for i in range(num_NN):
        m_dNN = EI * Dfai[:,i:i+1]
        v_dNN = dde.grad.jacobian(M, x, i = i, j=0)
        q_dNN = -dde.grad.jacobian(V, x, i = i, j=0)
        M_dNN.append(m_dNN)
        V_dNN.append(v_dNN)
        Q_dNN.append(q_dNN)
        Dy_t_dNN.append(dy_t_dNN)
        Dy_tt_dNN.append(dy_tt_dNN)
    loss_Fai = 0
    loss_Dfai = 0
    loss_M = 0
    loss_V = 0
    loss_Q = 0
    loss_EI = 0
    loss_Dy_t = 0
    loss_Dy_tt = 0
    
    loss_Fai = (Fai[:,0:1]-fai_dNN) * f_sig_mid(x_in,0) + (Fai[:,1:2]-fai_dNN) * f_sig_mid(x_in,1)    
    for i in range(num_NN):
        if i == 1:
            f_i = F/L_dis
            W_i = 0.1
        else:
            f_i = 0
            W_i =1
        Weight_M =  tf.sign (tf.tanh(tf.nn.relu( tf.abs(M[:,i:i+1]) - 1e-4)))
        loss_Dfai = loss_Dfai + (Dfai[:,i:i+1] - dfai_dNN) * f_sig0(x_in,i)
        loss_M = loss_M + (M[:,i:i+1]-M_dNN[i]) * f_sig0(x_in,i)
        loss_V = loss_V + (V[:,i:i+1]-V_dNN[i]) * f_sig0(x_in,i)
        loss_Dy_t = loss_Dy_t + (Dy_t-Dy_t_dNN[i]) * f_sig0(x_in,i) * f_T(t_in)
        loss_Q = loss_Q + ( rho*A*Dy_tt_dNN[i]/EI_real + Q_dNN[i] -f_i*tf.sin(w0*t_in)/EI_real ) * f_sig0(x_in,i)
    return [10*loss_Fai,
            loss_Dfai,
            loss_M,
            loss_V,
            loss_Dy_t,
            loss_Q]


def Boun_func1(x,y):
    dfai = dde.grad.jacobian(y,x, i =num_NN*1 , j= 0)
    return dfai
def Boun_func2(x,y):
    dfai = dde.grad.jacobian(y,x, i =num_NN*2 -1, j= 0)
    return dfai

geom = dde.geometry.Interval(0, L)
timedomain = dde.geometry.TimeDomain(0, T)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

#Set initial conditions
ic1 = dde.icbc.OperatorBC(geomtime,lambda x, y, _: dde.grad.hessian(y, x,component = 0,\
                                                                    i=1,j=1)/EI_real,lambda x, _: np.isclose(x[1], 0))
ic2 = dde.icbc.OperatorBC(geomtime,lambda x, y, _: dde.grad.hessian(y, x,component = 1,\
                                                                    i=1,j=1)/EI_real,lambda x, _: np.isclose(x[1], 0))
ic3 = dde.icbc.OperatorBC(geomtime,lambda x, y, _: dde.grad.hessian(y, x,component = 2,\
                                                                    i=1,j=1)/EI_real,lambda x, _: np.isclose(x[1], 0))

data = dde.data.TimePDE(geomtime,beampde2,[observe_u1,observe_u3],\
                        num_domain=num_domain, anchors=anchors_x)

# net = dde.maps.FNN([2] + [100]*3 +  [2+num_NN*4], "tanh", "Glorot uniform")
net = dde.maps.FNN([2] + [2], None, "Glorot uniform")

#Apply hard constraints for boundary and interface conditions
def modify_output(X, y):
    x = X[:,0:1]
    t = X[:,1:2]
    u     = y[:,0:2]
    Fai   = y[:,2:4]
    Dfai  = y[:,1+num_NN*1:1+num_NN*2]
    M     = y[:,1+num_NN*2:1+num_NN*3]
    V     = y[:,1+num_NN*3:1+num_NN*4]
    Dy_t  = y[:,1+num_NN*4:2+num_NN*4]
    
    y = c1/(c1+c2+c3)*tf.layers.dense(X, 50, tf.nn.tanh) + c2/(c1+c2+c3)*tf.layers.dense(X, 50, tf.sin)\
         + c3/(c1+c2+c3)*tf.layers.dense(X, 50, tf.nn.elu)
    for i in range(4):
        y = c1/(c1+c2+c3)*tf.layers.dense(y, 50, tf.nn.tanh) + c2/(c1+c2+c3)*tf.layers.dense(y, 50, tf.sin)\
         + c3/(c1+c2+c3)*tf.layers.dense(y, 50, tf.nn.elu)
    y = tf.layers.dense(y, num_NN*5+1, None)
    
    u     = y[:,0:2]
    Fai   = y[:,2:4]
    Dfai  = y[:,1+num_NN*1:1+num_NN*2]
    M     = y[:,1+num_NN*2:1+num_NN*3]
    V     = y[:,1+num_NN*3:1+num_NN*4]
    Dy_t  = y[:,1+num_NN*4:2+num_NN*4]
    
    Hard_BC = tf.layers.dense(t, 30, activation_func2)
    for i in range(2):
        Hard_BC = tf.layers.dense(Hard_BC, 30, activation_func2)
    Hard_BC = tf.layers.dense(Hard_BC, 8, None)
    
    EI = tf.layers.dense(x, E_nodenum, activation_func2)
    for i in range(2):
        EI = tf.layers.dense(EI, E_nodenum, activation_func2)
    EI = tf.layers.dense(EI, 1, None)

    
    final_output = tf.concat([0.01*t*(x*u[:,0:1]*(x-s_mid) + P1*Hard_BC[:,0:1]*x/s_mid),\
                              0.01*t*(u[:,1:2]*(x-L)*(x-s_mid) + P1*Hard_BC[:,0:1]*(x-L)/(s_mid-L)),\
                              0.01*t*(Fai[:,0:1]*(x-s_mid) + P2*Hard_BC[:,1:2]), \
                              0.01*t*(Fai[:,1:2]*(x-s_mid) + P2*Hard_BC[:,1:2]), \
                              0.01*t*(x*Dfai[:,0:1]*(x-s1)+P3*Hard_BC[:,2:3]*x/s1),\
                              0.01*t*(Dfai[:,1:2]*(x-s1)*(x-s2)+P3*Hard_BC[:,2:3]*(x-s2)/(s1-s2)+\
                                                                P4*Hard_BC[:,3:4]*(x-s1)/(s2-s1)),\
                              0.01*t*(Dfai[:,2:3]*(x-L)*(x-s2)+P4*Hard_BC[:,3:4]*(x-L)/(s2-L)),\
                              0.01*t*(x*M[:,0:1]*(x-s1)+P5*Hard_BC[:,4:5]*x/s1),\
                              0.01*t*(M[:,1:2]*(x-s1)*(x-s2)+P5*Hard_BC[:,4:5]*(x-s2)/(s1-s2)+P6*Hard_BC[:,5:6]*(x-s1)/(s2-s1)),\
                              0.01*t*(M[:,2:3]*(x-L)*(x-s2)+P6*Hard_BC[:,5:6]*(x-L)/(s2-L)),\
                              0.01*t*(V[:,0:1]*(x-s1)+P7*Hard_BC[:,6:7]),\
                              0.01*t*(V[:,1:2]*(x-s1)*(x-s2)+P7*Hard_BC[:,6:7]*(x-s2)/(s1-s2)+P8*Hard_BC[:,7:8]*(x-s1)/(s2-s1)),\
                              0.01*t*(V[:,2:3]*(x-s2)+P8*Hard_BC[:,7:8]),\
                              P0*t*(x*Dy_t*(x-L)),
                              tf.abs(EI)
                             ], axis=1)
    return final_output

net.apply_output_transform(modify_output)
variable = dde.callbacks.VariableValue(SS, period=200, filename="s1.dat")#200为epochs之间的间隔
variable2 = dde.callbacks.VariableValue(List_P, period=200, filename="w1.dat")#200为epochs之间的间隔
model = dde.Model(data, net)

Num_loss = 8

loss = []
Weight = np.ones(Num_loss)
Weight[-2:] = 100
for i in range(Num_loss):
    loss.append('MAE')
    # if i <13:
    #     if i%2==1:
    #         loss[i] = 'MSE'
print(loss)
#     if 1<i<6:
#         loss[i] = 'zero'
# loss[4] = 'MAE'
total_time = 0
start_time = time.time()
model.compile("adam", lr, loss_weights=list(Weight.ravel()), loss = loss)
losshistory, train_state = model.train(epochs=20000, callbacks=[variable,variable2])
end_time = time.time()
total_time = total_time + end_time - start_time
list_NN = []
list_points = []
list_reslut = []

NN = Output(XY1[:,0:2])
NN = np.array(NN).reshape(len(NN),-1).T
EI = NN[:,-1:]
norm_EI = np.linalg.norm((EI-NP_EI(X1)),ord=2) / np.linalg.norm(NP_EI(X1),ord=2)
list_NN.append(np.array(NN).reshape(len(NN),-1).T)
list_reslut.append(norm_EI)
print('Norm_EI',norm_EI)

In [ ]:
#Reduce the learning rate
model.compile("adam", 1e-4, loss_weights=list(Weight.ravel()), loss = loss)
losshistory, train_state = model.train(epochs=20000, callbacks=[variable,variable2])
NN = Output(XY1[:,0:2])
NN = np.array(NN).reshape(len(NN),-1).T
EI = NN[:,-1:]
norm_EI = np.linalg.norm((EI-NP_EI(X1)),ord=2) / np.linalg.norm(NP_EI(X1),ord=2)
list_NN.append(np.array(NN).reshape(len(NN),-1).T)
list_reslut.append(norm_EI)
print('Norm_EI',norm_EI)

In [ ]:
for i in range(10):
    # model.compile("adam", 1e-4, loss_weights=list(Weight.ravel()), loss = loss)
    losshistory, train_state = model.train(epochs=20000, callbacks=[variable,variable2])
    NN = Output(XY1[:,0:2])
    NN = np.array(NN).reshape(len(NN),-1).T
    EI = NN[:,-1:]
    norm_EI = np.linalg.norm((EI-NP_EI(X1)),ord=2) / np.linalg.norm(NP_EI(X1),ord=2)
    list_NN.append(np.array(NN).reshape(len(NN),-1).T)
    list_reslut.append(norm_EI)
    print('Norm_EI',norm_EI)

In [ ]:
for i in range(13):
    # model.compile("adam", 1e-4, loss_weights=list(Weight.ravel()), loss = loss)
    losshistory, train_state = model.train(epochs=20000, callbacks=[variable,variable2])
    NN = Output(XY1[:,0:2])
    NN = np.array(NN).reshape(len(NN),-1).T
    EI = NN[:,-1:]
    norm_EI = np.linalg.norm((EI-NP_EI(X1)),ord=2) / np.linalg.norm(NP_EI(X1),ord=2)
    list_NN.append(np.array(NN).reshape(len(NN),-1).T)
    list_reslut.append(norm_EI)
    print('Norm_EI',norm_EI)

In [ ]:
losshistory, train_state = model.train(epochs=23106, callbacks=[variable,variable2])
NN = Output(XY1[:,0:2])
NN = np.array(NN).reshape(len(NN),-1).T
EI = NN[:,-1:]
norm_EI = np.linalg.norm((EI-NP_EI(X1)),ord=2) / np.linalg.norm(NP_EI(X1),ord=2)
list_NN.append(np.array(NN).reshape(len(NN),-1).T)
list_reslut.append(norm_EI)
print('Norm_EI',norm_EI)